<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [29]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [30]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [31]:
# placeholder 정의
x = tf.placeholder('float', shape=[None, 784])
y_true = tf.placeholder('float', shape=[None, 10])

x_img = tf.reshape(x, [-1, 28, 28, 1])  # 784 = 28 * 28
x_img.get_shape()

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

* 합성곱 계층
    * feature map의 집합을 말한다.
    * 각 feature map(kernel, filter)는 하나의 특징을 검출한다.
    * cnn은 입력 계층의 일부 데이터만 은닉 계층에 연결
    * w, b(kernel, filter, feature map) 를 모든 은닉계층이 공유 => 가중치 매개 변수 감소
    * 5*5 window 사용:  28 * 28 => 24 * 24(28-5+1)
    * stride: 윈도우가 한번에 움직이는 거리
    * padding: 테두리의 크기
* 풀링 계층
    * 데이터 압축
    * ex: max pooling은 2x2 영역에서 가장 큰 값을 선택해 정보를 압축한다.
    * 특징의 정확한 위치보다는 다른 특징과의 상대적 위치가 중요함.
* 차원 변화
    * 입력 계층(28x28)
    * 합성곱 계층(None x 24 x 24, window=5, stride=1)
    * 풀링 계층(None x 12 x 12, maxpulling) 


In [33]:
def weight_variable(shape):
    # w 초기화
    # truncated normal: 정규분포를 따르는 수열 생성, 2sigma 이상인 값은 제외
    init = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(init)


def bias_variable(shape):
    # b 초기화, 0.1
    init = tf.constant(.1, shape=shape)
    return tf.Variable(init)


def conv2d(x, w):
    # 합성곱 계층
    # stride: The stride of the sliding window for each dimension of input
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    # ksize: 커널 크기 2x2
    # strides: 2x2
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [34]:
## layer 1
# window: 5x5, 32개 필터 정의
# 세번째 원소는 color
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_img, w_conv1) + b_conv1) # relu
h_pool1 = max_pool_2x2(h_conv1)  # pooling

## layer2
# 32: conv1에서 출력 크기
# window: 5x5, 64개 필터 정의
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


## output
# 1024뉴런
w_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])  # 직렬화
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder('float')  # 드롭아웃 되지않을 확률
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# softmax
w_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])  # 0-10
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

In [42]:
## train model

with tf.name_scope('loss'):
    cross_entropy = -tf.reduce_sum(y_true * tf.log(y_conv))  # loss function
    train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)  # optimizer
    tf.summary.scalar('loss', cross_entropy)

with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_true, 1))  # y_true is one-hot encoded
    accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))  # metric: accuracy
    tf.summary.scalar('accuracy', accuracy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())  # 변수 초기화

writer = tf.summary.FileWriter('./logs/cnn_test')
writer.add_graph(sess.graph)
merged = tf.summary.merge_all()

for i in range(100):
    batch = mnist.train.next_batch(100)  # 100개씩 배치학습
    summary, train_accuracy = sess.run([merged, accuracy], feed_dict={x: batch[0], y_true: batch[1], keep_prob: 1})
    sess.run(train_step, feed_dict={x: batch[0], y_true: batch[1], keep_prob: 0.5})
    writer.add_summary(summary, i)
    if i%10 == 0:
        print(f' iter {i} training accuracy: {train_accuracy}')
print('end')

 iter 0 training accuracy: 0.05000000074505806
 iter 10 training accuracy: 0.20000000298023224
 iter 20 training accuracy: 0.5
 iter 30 training accuracy: 0.6100000143051147
 iter 40 training accuracy: 0.6800000071525574
 iter 50 training accuracy: 0.8299999833106995
 iter 60 training accuracy: 0.8100000023841858
 iter 70 training accuracy: 0.8700000047683716
 iter 80 training accuracy: 0.8100000023841858
 iter 90 training accuracy: 0.8399999737739563
end


In [40]:
test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_true: mnist.test.labels, keep_prob: 1})
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.8870000243186951


In [41]:
sess.close()